In [2]:
! pip install neo4j

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.6/296.6 kB 6.5 MB/s eta 0:00:00a 0:00:01


In [27]:
! pip install tqdm

In [29]:
from neo4j import GraphDatabase
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

# Define the connection URI and authentication
uri = "bolt://94.230.217.43:7687" #"bolt://localhost:7687" #"bolt://94.230.217.43:7687"  # Update this if you're using a different address   localhost
username = "neo4j"  # Your Neo4j username
password = "lagginhorn" #"lagginhorn"  # Your Neo4j password   newpassword

# Create a Neo4j driver instance
driver = GraphDatabase.driver(uri, auth=(username, password))

### Utils

In [32]:
def get_node_data_with_relation_and_node(tx, node_label, relation, node_label_2):
    query = """
    MATCH (b:{})-[r:{}]->(rdata:{})
    RETURN b, rdata LIMIT 100000
    """.format(node_label, relation, node_label_2)
    result = tx.run(query)
    return [{node_label: record["b"], node_label_2: record["rdata"]} for record in result]

# Main block to run the query
def get_node2_data_with_relation_and_node_data_main(node_label="Person", relation="SPONSORS", node_label_2="Bill"):
    with driver.session() as session:
        examples = session.read_transaction(get_node_data_with_relation_and_node, node_label, relation, node_label_2)
        # Output the examples
        df = pd.DataFrame(columns = [node_label_2, node_label])
        print("Node2 {} data".format(node_label_2), examples[0][node_label_2])
        print("Node1 {} data".format(node_label), examples[0][node_label])
        for example in examples:
            df.loc[len(df)] = [example[node_label_2], example[node_label]]
    return df


def get_one_node_data_with_relation_and_node(tx, node_label, relation, node_label_2, element_id):
    query = """
    MATCH (b:{})-[r:{}]->(rdata:{})
    where elementId(b) = '{}'
    RETURN b, rdata LIMIT 1000
    """.format(node_label, relation, node_label_2, element_id)
    result = tx.run(query)
    return [{node_label: record["b"], node_label_2: record["rdata"]} for record in result]

# Main block to run the query
def get_one_node_data_with_relation_and_node_main(element_id, node_label="Person", relation="REPRESENTS", node_label_2="Canton"):
    with driver.session() as session:
        examples = session.read_transaction(get_one_node_data_with_relation_and_node, node_label, relation, node_label_2, element_id)
        # Output the examples
        # print("Node1 {} data".format(node_label), examples[0][node_label])
        # print("Node2 {} data".format(node_label_2), examples[0][node_label_2])
    if examples:
       return examples[0][node_label_2]
    #print("NOT FOUND")
    return "NOT FOUND"

In [33]:
# Get 10000 Bill with Person Sponsors
df_bill_person = get_node2_data_with_relation_and_node_data_main(node_label="Person", relation="SPONSORS", node_label_2="Bill")
df_bill_person["Bill"] = df_bill_person["Bill"].apply(lambda l : {"element_id": l.element_id, "bill_number": l['bill_number'], "title_fr": l['title_fr'], "title_de": l['title_de'], "urgent": l['urgent']})
df_bill_person["Person"] = df_bill_person["Person"].apply(lambda l : {"element_id": l.element_id, "last_name": l['last_name'], "first_name":l["first_name"], "gender": l['gender']})

# Associate Canton to Person
df_bill_person["Canton"] = df_bill_person["Person"].progress_apply(lambda l: get_one_node_data_with_relation_and_node_main(l["element_id"]))
df_bill_person["Canton"] = df_bill_person["Canton"].apply(lambda l : {"name": l['name'], "abbrev": l['abbrev']})

# Get Tag for each Bill  
df_bill_person["Tag"] = df_bill_person["Bill"].progress_apply(lambda l: get_one_node_data_with_relation_and_node_main(l["element_id"], node_label="Bill", relation="ABOUT", node_label_2="Tag"))
df_bill_person = df_bill_person[df_bill_person["Tag"]!="NOT FOUND"]
df_bill_person["Tag"] = df_bill_person["Tag"].apply(lambda l : {"name": l['name']})

# Associate Debates
df_bill_person["Debate"] = df_bill_person["Bill"].progress_apply(lambda l: get_one_node_data_with_relation_and_node_main(l["element_id"], node_label="Bill", relation="CONTAINS", node_label_2="Debate"))
df_bill_person["Debate"] = df_bill_person["Debate"].apply(lambda l : {"date_start": l['date_start']})

/tmp/ipykernel_1161/1591477151.py:12: DeprecationWarning: read_transaction has been renamed to execute_read
  examples = session.read_transaction(get_node_data_with_relation_and_node, node_label, relation, node_label_2)


Node2 Bill data <Node element_id='4:fa996e42-3a3f-4866-ad85-cb06ad63f099:454275' labels=frozenset({'Question Time', 'Bill'}) properties={'title_de': 'Kyoto-Protokoll. Bilanz der bisherigen Politik?', 'bill_number': '09.5595', 'title_fr': 'Protocole de Kyoto. Bilan de la politique menée?', 'urgent': False, 'status': 'Resolved'}>
Node1 Person data <Node element_id='4:fa996e42-3a3f-4866-ad85-cb06ad63f099:585934' labels=frozenset({'Person', 'Unique Person'}) properties={'date_birth': neo4j.time.Date(1969, 5, 5), 'uid': 3888, 'gender': 'm', 'last_name': 'Hiltpold', 'first_name': 'Hugues'}>


  0%|          | 0/31823 [00:00<?, ?it/s]/tmp/ipykernel_1161/1591477151.py:34: DeprecationWarning: read_transaction has been renamed to execute_read
  examples = session.read_transaction(get_one_node_data_with_relation_and_node, node_label, relation, node_label_2, element_id)
  0%|          | 0/31823 [00:00<?, ?it/s]/tmp/ipykernel_1161/1591477151.py:34: DeprecationWarning: read_transaction has been renamed to execute_read
  examples = session.read_transaction(get_one_node_data_with_relation_and_node, node_label, relation, node_label_2, element_id)
100%|██████████| 31823/31823 [09:33<00:00, 55.52it/s]


In [38]:
df_bill_person["Debate"] = df_bill_person["Bill"].progress_apply(lambda l: get_one_node_data_with_relation_and_node_main(l["element_id"], node_label="Bill", relation="CONTAINS", node_label_2="Debate"))

  0%|          | 0/31737 [00:00<?, ?it/s]/tmp/ipykernel_1161/1591477151.py:34: DeprecationWarning: read_transaction has been renamed to execute_read
  examples = session.read_transaction(get_one_node_data_with_relation_and_node, node_label, relation, node_label_2, element_id)
100%|██████████| 31737/31737 [09:29<00:00, 55.73it/s]


In [40]:
df_bill_person["Debate"] = df_bill_person["Debate"].apply(lambda l : {"date_start": l['date_start']})

In [41]:
df_bill_person

,Bill,Person,Canton,Tag,Debate
0,{'element_id': '4:fa996e42-3a3f-4866-ad85-cb06...,{'element_id': '4:fa996e42-3a3f-4866-ad85-cb06...,"{'name': 'Genf', 'abbrev': 'GE'}",{'name': 'Umwelt'},{'date_start': 2009-12-07}
1,{'element_id': '4:fa996e42-3a3f-4866-ad85-cb06...,{'element_id': '4:fa996e42-3a3f-4866-ad85-cb06...,"{'name': 'Zürich', 'abbrev': 'ZH'}",{'name': 'Medien und Kommunikation'},{'date_start': 2009-12-07}
2,{'element_id': '4:fa996e42-3a3f-4866-ad85-cb06...,{'element_id': '4:fa996e42-3a3f-4866-ad85-cb06...,"{'name': 'Aargau', 'abbrev': 'AG'}",{'name': 'Internationale Politik'},{'date_start': 2009-12-07}
3,{'element_id': '4:fa996e42-3a3f-4866-ad85-cb06...,{'element_id': '4:fa996e42-3a3f-4866-ad85-cb06...,"{'name': 'Bern', 'abbrev': 'BE'}",{'name': 'Verkehr'},{'date_start': 2009-12-07}
4,{'element_id': '4:fa996e42-3a3f-4866-ad85-cb06...,{'element_id': '4:fa996e42-3a3f-4866-ad85-cb06...,"{'name': 'Zürich', 'abbrev': 'ZH'}",{'name': 'Recht Allgemein'},{'date_start': 2009-12-07}
...,...,...,...,...,...
31818,{'element_id': '4:fa996e42-3a3f-4866-ad85-cb06...,{'element_id': '4:fa996e42-3a3f-4866-ad85-cb06...,"{'name': 'Zürich', 'abbrev': 'ZH'}",{'name': 'Europapolitik'},{'date_start': 2008-06-09}
31819,{'element_id': '4:fa996e42-3a3f-4866-ad85-cb06...,{'element_id': '4:fa996e42-3a3f-4866-ad85-cb06...,"{'name': 'Bern', 'abbrev': 'BE'}",{'name': 'Finanzwesen'},{'date_start': 2008-12-15}
31820,{'element_id': '4:fa996e42-3a3f-4866-ad85-cb06...,{'element_id': '4:fa996e42-3a3f-4866-ad85-cb06...,"{'name': 'Zürich', 'abbrev': 'ZH'}",{'name': 'Europapolitik'},{'date_start': 2008-09-29}
31821,{'element_id': '4:fa996e42-3a3f-4866-ad85-cb06...,{'element_id': '4:fa996e42-3a3f-4866-ad85-cb06...,"{'name': 'Zürich', 'abbrev': 'ZH'}",{'name': 'Internationale Politik'},{'date_start': 2008-09-22}


In [21]:
df_bill_person

,Bill,Person,Canton,Tag
0,{'element_id': '4:fa996e42-3a3f-4866-ad85-cb06...,{'element_id': '4:fa996e42-3a3f-4866-ad85-cb06...,"{'name': 'Genf', 'abbrev': 'GE'}",{'name': 'Umwelt'}
1,{'element_id': '4:fa996e42-3a3f-4866-ad85-cb06...,{'element_id': '4:fa996e42-3a3f-4866-ad85-cb06...,"{'name': 'Zürich', 'abbrev': 'ZH'}",{'name': 'Medien und Kommunikation'}
2,{'element_id': '4:fa996e42-3a3f-4866-ad85-cb06...,{'element_id': '4:fa996e42-3a3f-4866-ad85-cb06...,"{'name': 'Aargau', 'abbrev': 'AG'}",{'name': 'Internationale Politik'}
3,{'element_id': '4:fa996e42-3a3f-4866-ad85-cb06...,{'element_id': '4:fa996e42-3a3f-4866-ad85-cb06...,"{'name': 'Bern', 'abbrev': 'BE'}",{'name': 'Verkehr'}
4,{'element_id': '4:fa996e42-3a3f-4866-ad85-cb06...,{'element_id': '4:fa996e42-3a3f-4866-ad85-cb06...,"{'name': 'Zürich', 'abbrev': 'ZH'}",{'name': 'Recht Allgemein'}
...,...,...,...,...
9995,{'element_id': '4:fa996e42-3a3f-4866-ad85-cb06...,{'element_id': '4:fa996e42-3a3f-4866-ad85-cb06...,"{'name': 'Genf', 'abbrev': 'GE'}",{'name': 'Sozialer Schutz'}
9996,{'element_id': '4:fa996e42-3a3f-4866-ad85-cb06...,{'element_id': '4:fa996e42-3a3f-4866-ad85-cb06...,"{'name': 'Basel-Stadt', 'abbrev': 'BS'}",{'name': 'Gesundheit'}
9997,{'element_id': '4:fa996e42-3a3f-4866-ad85-cb06...,{'element_id': '4:fa996e42-3a3f-4866-ad85-cb06...,"{'name': 'Bern', 'abbrev': 'BE'}",{'name': 'Kultur'}
9998,{'element_id': '4:fa996e42-3a3f-4866-ad85-cb06...,{'element_id': '4:fa996e42-3a3f-4866-ad85-cb06...,"{'name': 'St. Gallen', 'abbrev': 'SG'}",{'name': 'Landwirtschaft'}


## Use cases views

In [49]:
df_uc_1 = df_bill_person.copy()
df_uc_1["bill_id"] = df_uc_1["Bill"].apply(lambda l : l["element_id"])
df_uc_1["bill_title_fr"] = df_uc_1["Bill"].apply(lambda l : l["title_fr"])
df_uc_1["bill_title_de"] = df_uc_1["Bill"].apply(lambda l : l["title_de"])

df_uc_1["person_name"] = df_uc_1["Person"].apply(lambda l : l["last_name"] + " " + l["first_name"])
df_uc_1["person_gender"] = df_uc_1["Person"].apply(lambda l : l["gender"])

df_uc_1["canton_name"] = df_uc_1["Canton"].apply(lambda l : l["name"])
df_uc_1["canton_abbrev"] = df_uc_1["Canton"].apply(lambda l : l["abbrev"])

df_uc_1["topic"] = df_uc_1["Tag"].apply(lambda l : l["name"])

df_uc_1["date_start"] = df_uc_1["Debate"].apply(lambda l : str(l["date_start"]))

df_uc_1.drop(columns = ["Bill", "Person", "Canton", "Tag", "Debate"], inplace=True)
df_uc_1.drop_duplicates(["bill_id"], inplace = True)
df_uc_1

,bill_id,bill_title_fr,bill_title_de,person_name,person_gender,canton_name,canton_abbrev,topic,date_start
0,4:fa996e42-3a3f-4866-ad85-cb06ad63f099:454275,Protocole de Kyoto. Bilan de la politique menée?,Kyoto-Protokoll. Bilanz der bisherigen Politik?,Hiltpold Hugues,m,Genf,GE,Umwelt,2009-12-07
1,4:fa996e42-3a3f-4866-ad85-cb06ad63f099:454276,Possibilités d'économies pour la SSR,Einsparungspotenzial bei der SRG,Bäumle Martin,m,Zürich,ZH,Medien und Kommunikation,2009-12-07
2,4:fa996e42-3a3f-4866-ad85-cb06ad63f099:454277,Coup d'Etat au Honduras,Putsch in Honduras,Müller Geri,m,Aargau,AG,Internationale Politik,2009-12-07
3,4:fa996e42-3a3f-4866-ad85-cb06ad63f099:454278,Premières chutes de neige et gestion de la cir...,Wintereinbruch und Verkehrsmanagement auf der A2,Brönnimann Andreas,m,Bern,BE,Verkehr,2009-12-07
4,4:fa996e42-3a3f-4866-ad85-cb06ad63f099:454279,Interdiction d'exercer une profession pour les...,Berufsverbot für Frauen,Kaufmann Hans,m,Zürich,ZH,Recht Allgemein,2009-12-07
...,...,...,...,...,...,...,...,...,...
31818,4:fa996e42-3a3f-4866-ad85-cb06ad63f099:732351,Explosion des coûts liés aux accords de Schengen,Explosion der Schengen-Kosten,Fehr Hans,m,Zürich,ZH,Europapolitik,2008-06-09
31819,4:fa996e42-3a3f-4866-ad85-cb06ad63f099:732352,Crise des marchés financiers 2. Assurances,Finanzmarktkrise 2. Versicherungen,Stöckli Hans,m,Bern,BE,Finanzwesen,2008-12-15
31820,4:fa996e42-3a3f-4866-ad85-cb06ad63f099:732353,Introduction de REACH. Enregistrement jusqu'au...,Einführung Reach. Registrierung bis zum 30. No...,Moser Tiana Angelina,f,Zürich,ZH,Europapolitik,2008-09-29
31821,4:fa996e42-3a3f-4866-ad85-cb06ad63f099:732354,"La Suisse, quartier général des FARC en Europe",Die Schweiz als europäisches Hauptquartier der...,Fehr Hans,m,Zürich,ZH,Internationale Politik,2008-09-22


In [23]:
! ls /home/jovyan/work/5-2-lagginhorn/

 democrasci.db		  neo4j-community-5.24.2-unix.tar.gz   test.csv
 democrasci_rdb.db	  neo4j.dump			       Untitled1.ipynb
 neo4j_5.24.2_all.deb	 'old - neo4j-community-5.24.2-old'    Untitled.ipynb
 neo4j-community-5.24.2   test_2.csv


In [24]:
df_uc_1.to_csv("/home/jovyan/work/5-2-lagginhorn/use_case_views/uc1_967.csv")

In [26]:
df_uc_1.to_csv("/home/jovyan/work/5-2-lagginhorn/use_case_views/uc1_9713.csv")

In [36]:
df_uc_1.to_csv("/home/jovyan/work/5-2-lagginhorn/use_case_views/uc1_31052.csv")

In [50]:
df_uc_1.to_csv("/home/jovyan/work/5-2-lagginhorn/use_case_views/uc3_31052.csv")

In [48]:
str(df_uc_1["date_start"][0])

'2009-12-07'

In [16]:
len(set(df_uc_1["topic"]))

20

In [5]:
df_bill_person["Bill"][0]

{'element_id': '4:fa996e42-3a3f-4866-ad85-cb06ad63f099:454275',
 'bill_number': '09.5595',
 'title_fr': 'Protocole de Kyoto. Bilan de la politique menée?',
 'title_de': 'Kyoto-Protokoll. Bilanz der bisherigen Politik?',
 'urgent': False}

In [6]:
df_bill_person["Person"][0]

{'element_id': '4:fa996e42-3a3f-4866-ad85-cb06ad63f099:585934',
 'last_name': 'Hiltpold',
 'first_name': 'Hugues',
 'gender': 'm'}

In [ ]:
"bill_id", "bill_title_fr", "bill_title_de", "person_name", "person_gender", "canton_name", "canton_abbrev", "topic"

In [9]:
get_unique_relationships_main("Person")

/tmp/ipykernel_243/4155827826.py:40: DeprecationWarning: read_transaction has been renamed to execute_read
  unique_relationships = session.read_transaction(get_unique_relationships, node_label)


Unique relationships related to 'Person' nodes:
BORN_IN
GAVE
PART_OF
ACTING_AS
COSPONSORS
SPONSORS
ACTIVE_DURING
CITIZEN_IN
ELECTED_TO
MEMBER_OF
REPRESENTS
ASSOCIATED_WITH
VOTED
WORKS_FOR
CANDIDATE


In [13]:
get_unique_relationships_with_nodes_main("Person")

/tmp/ipykernel_243/603547608.py:57: DeprecationWarning: read_transaction has been renamed to execute_read
  unique_relationships = session.read_transaction(get_unique_relationships_with_nodes, node_label)


Unique relationships related to 'Person' nodes:
('BORN_IN', ['City', 'Location'])
('GAVE', ['Transcript', 'Speech'])
('PART_OF', ['Committee'])
('PART_OF', ['Delegation'])
('ACTING_AS', ['Party', 'Organisation'])
('ACTING_AS', ['Committee'])
('ACTING_AS', ['Delegation'])
('COSPONSORS', ['Bill', 'Motion'])
('COSPONSORS', ['Bill', 'Postulate'])
('COSPONSORS', ['Bill', 'Interpellation'])
('COSPONSORS', ['Bill', 'Parliamentary Initiative'])
('SPONSORS', ['Bill', 'Question'])
('SPONSORS', ['Bill', 'Motion'])
('SPONSORS', ['Bill', 'Parliamentary Initiative'])
('SPONSORS', ['Bill', 'Postulate'])
('SPONSORS', ['Bill', 'Interpellation'])
('ACTIVE_DURING', ['Year'])
('CITIZEN_IN', ['City', 'Location'])
('ELECTED_TO', ['Chamber'])
('MEMBER_OF', ['Party', 'Organisation'])
('MEMBER_OF', ['Parliamentary Group'])
('REPRESENTS', ['Canton', 'Location'])
('ASSOCIATED_WITH', ['Organisation', 'Interest Group', 'Postprocessing Necessary', 'Foundation'])
('ASSOCIATED_WITH', ['Organisation', 'Interest Group'

In [10]:
from neo4j import GraphDatabase
import pandas as pd

# Define the connection URI and authentication
uri = "bolt://94.230.217.43:7687" #"bolt://localhost:7687" #"bolt://94.230.217.43:7687"  # Update this if you're using a different address   localhost
username = "neo4j"  # Your Neo4j username
password = "lagginhorn" #"lagginhorn"  # Your Neo4j password   newpassword

# Create a Neo4j driver instance
driver = GraphDatabase.driver(uri, auth=(username, password))

def get_unique_node_labels():
    with driver.session() as session:
        # Cypher query to get all labels of nodes
        result = session.run("MATCH (n) RETURN DISTINCT labels(n) AS label_list")
        # Extracting and flattening the label lists
        unique_labels = set()
        for record in result:
            unique_labels.update(record["label_list"])
        return unique_labels

# Fetch and print the unique node labels
def get_unique_node_labels_main():
    unique_labels = get_unique_node_labels()
    for label in unique_labels:
        print(label)


def get_unique_relationships(tx, node_label):
    query = """
    MATCH (b:{})-[r]-()
    RETURN DISTINCT type(r) AS relationship
    """.format(node_label)
    result = tx.run(query)    
    return [record["relationship"] for record in result]

# Main block to run the query
def get_unique_relationships_main(node_label="Bill"):
    with driver.session() as session:
        unique_relationships = session.read_transaction(get_unique_relationships, node_label)
        print("Unique relationships related to '{}' nodes:".format(node_label))
        for rel in unique_relationships:
            print(rel)


def get_unique_relationships_with_nodes(tx, node_label):
    query = """
    MATCH (b:{})-[r]->(n)
    RETURN DISTINCT type(r) AS RelationshipType, labels(n) AS RelatedNodeLabels
    """.format(node_label)
    result = tx.run(query)    
    return [(record["RelationshipType"], record["RelatedNodeLabels"]) for record in result]

# Main block to run the query
def get_unique_relationships_with_nodes_main(node_label="Bill"):
    with driver.session() as session:
        unique_relationships = session.read_transaction(get_unique_relationships_with_nodes, node_label)
        print("Unique relationships related to '{}' nodes:".format(node_label))
        for rel in unique_relationships:
            print(rel)


def get_node_data(tx, node_label):
    query = """
    MATCH (b:{})
    RETURN b LIMIT 100
    """.format(node_label)
    result = tx.run(query)
    return [record["b"] for record in result]

# Main block to run the query
def get_node_data_main(node_label="Bill"):
    list_nodes = []
    with driver.session() as session:
        examples = session.read_transaction(get_node_data, node_label)
        # Output the examples
        for example in examples:
            print(example)
            print("-" * 40)
            list_nodes.append(example)
    return list_nodes


def get_node_data_with_properties(tx, node_label, property_name):
    query = """
    MATCH (b:{})
    WHERE b.name = '{}'
    RETURN b LIMIT 10
    """.format(node_label, property_name)
    result = tx.run(query)
    return [record["b"] for record in result]

# Main block to run the query
def get_node_data_with_properties_main(node_label="Tag", property_name="Verkehr"):
    with driver.session() as session:
        examples = session.read_transaction(get_node_data_with_properties, node_label, property_name)
        # Output the examples
        for example in examples:
            print(example)
            print("-" * 40)


def get_node_data_with_relation(tx, node_label, relation):
    query = """
    MATCH (b:{})-[r:{}]->(rdata)
    RETURN b, rdata LIMIT 10
    """.format(node_label, relation)
    result = tx.run(query)
    return [{node_label: record["b"], relation: record["rdata"]} for record in result]

# Main block to run the query
def get_node_data_with_relation_data_main(node_label="Bill", relation="ABOUT"):
    with driver.session() as session:
        examples = session.read_transaction(get_node_data_with_relation, node_label, relation)
        # Output the examples
        for example in examples:
            print("Node {} data".format(node_label), example[node_label])
            print("Relation {} data".format(relation), example[relation])
            print("-" * 40)


def get_node_data_with_relation_and_node(tx, node_label, relation, node_label_2):
    query = """
    MATCH (b:{})-[r:{}]->(rdata:{})
    RETURN b, rdata LIMIT 10
    """.format(node_label, relation, node_label_2)
    result = tx.run(query)
    return [{node_label: record["b"], node_label_2: record["rdata"]} for record in result]

# Main block to run the query
def get_node_data_with_relation_and_node_data_main(node_label="Bill", relation="ABOUT", node_label_2="Tag"):
    with driver.session() as session:
        examples = session.read_transaction(get_node_data_with_relation_and_node, node_label, relation, node_label_2)
        # Output the examples
        for example in examples:
            print("Node1 {} data".format(node_label), example[node_label])
            print("{}, Node2 {} data".format(relation, node_label_2), example[node_label_2])
            print("-" * 40)


def get_bill_with_specific_about_tag(tx, node_label, relation, node_label_2):
    query = """
    MATCH (b:{})-[r:{}]->(rdata:{})
    where rdata.name IN ["Gesundheit", "Umwelt", "Bildung", "Wirtschaft", "Wissenschaft und Forschung"]
    RETURN b, rdata LIMIT 10000
    """.format(node_label, relation, node_label_2)
    result = tx.run(query)
    return [{node_label: record["b"], node_label_2: record["rdata"]} for record in result]

# Main block to run the query
def get_bill_with_specific_about_tag_main(node_label="Bill", relation="ABOUT", node_label_2="Tag"):
    with driver.session() as session:
        examples = session.read_transaction(get_bill_with_specific_about_tag, node_label, relation, node_label_2)
        # Output the examples
        df = pd.DataFrame(columns = ["Bill", "Tag"])
        print("Node1 {} data".format(node_label), examples[0][node_label])
        print("{}, Node2 {} data".format(relation, node_label_2), examples[0][node_label_2])
        for example in examples:
            df.loc[len(df)] = [example[node_label], example[node_label_2]]
    return df
            


# Health, Environment, Education, Economy, Science research
# Gesundheit, Umwelt, Bildung, Wirtschaft, Wissenschaft und Forschung

In [4]:

#### REFAIRE AVEC PERSON BILL

def get_node_data_with_relation_and_node(tx, node_label, relation, node_label_2):
    query = """
    MATCH (b:{})-[r:{}]->(rdata:{})
    RETURN b, rdata LIMIT 1000
    """.format(node_label, relation, node_label_2)
    result = tx.run(query)
    return [{node_label: record["b"], node_label_2: record["rdata"]} for record in result]

# Main block to run the query
def get_node_data_with_relation_and_node_data_main(node_label="Canton", relation="SPONSORS", node_label_2="Bill"):
    with driver.session() as session:
        examples = session.read_transaction(get_node_data_with_relation_and_node, node_label, relation, node_label_2)
        # Output the examples
        df = pd.DataFrame(columns = [node_label, node_label_2])
        print("Node1 {} data".format(node_label_2), examples[0][node_label_2])
        print("Node2 {} data".format(node_label), examples[0][node_label])
        for example in examples:
            df.loc[len(df)] = [example[node_label_2], example[node_label]]
    return df

df_bill_canton = get_node_data_with_relation_and_node_data_main()
df_bill_canton["Bill"] = df_bill_canton["Bill"].apply(lambda l : {"element_id": l.element_id, "bill_number": l['bill_number'], "title_fr": l['title_fr'], "title_de": l['title_de']})
df_bill_canton["Canton"] = df_bill_canton["Canton"].apply(lambda l : {"name": l['name'], "abbrev": l['abbrev']})

/tmp/ipykernel_243/1659919274.py:14: DeprecationWarning: read_transaction has been renamed to execute_read
  examples = session.read_transaction(get_node_data_with_relation_and_node, node_label, relation, node_label_2)


Node1 Bill data <Node element_id='4:fa996e42-3a3f-4866-ad85-cb06ad63f099:693723' labels=frozenset({'Cantonal Initiative', 'Bill'}) properties={'title_de': 'Einbürgerungsverfahren', 'bill_number': '03.317', 'title_fr': 'Procédure de naturalisation', 'urgent': False, 'status': 'Resolved'}>
Node2 Canton data <Node element_id='4:fa996e42-3a3f-4866-ad85-cb06ad63f099:518263' labels=frozenset({'Canton', 'Location'}) properties={'name': 'Schwyz', 'abbrev': 'SZ'}>


In [113]:
def get_bill_with_about_tag(tx, node_label, relation, node_label_2, element_id):
    query = """
    MATCH (b:{})-[r:{}]->(rdata:{})
    where elementId(b) = '{}'
    RETURN rdata LIMIT 10000
    """.format(node_label, relation, node_label_2, element_id)
    result = tx.run(query)
    return [{node_label_2: record["rdata"]} for record in result]

# Main block to run the query
def get_bill_with_about_tag_main(element_id, node_label="Bill", relation="ABOUT", node_label_2="Tag"):
    with driver.session() as session:
        examples = session.read_transaction(get_bill_with_about_tag, node_label, relation, node_label_2, element_id)
        # Output the examples
        tag_value = examples[0][node_label_2]
    return tag_value

df_bill_canton["Tag"] = df_bill_canton["Bill"].apply(lambda l: get_bill_with_about_tag_main(l["element_id"])["name"])

In [123]:
df_bill_canton["Tag"] = df_bill_canton["Bill"].apply(lambda l: get_bill_with_about_tag_main(l["element_id"])["name"])

/tmp/ipykernel_8066/624684903.py:13: DeprecationWarning: read_transaction has been renamed to execute_read
  examples = session.read_transaction(get_bill_with_about_tag, node_label, relation, node_label_2, element_id)


In [129]:
df_bill_canton["BillText"] = df_bill_canton["Bill"].apply(lambda l: get_bill_with_about_tag_main(l["element_id"], relation="CONTAINS", node_label_2="BillText")["text"])

/tmp/ipykernel_8066/624684903.py:13: DeprecationWarning: read_transaction has been renamed to execute_read
  examples = session.read_transaction(get_bill_with_about_tag, node_label, relation, node_label_2, element_id)


In [133]:
def get_person_of_bill(tx, node_label, relation, node_label_2, element_id):
    query = """
    MATCH (b:{})-[r:{}]->(rdata:{})
    where elementId(rdata) = '{}'
    RETURN b LIMIT 10000
    """.format(node_label, relation, node_label_2, element_id)
    result = tx.run(query)
    return [{node_label: record["b"]} for record in result]

# Main block to run the query
def get_person_of_bill_main(element_id, node_label="Person", relation="SPONSORS", node_label_2="Bill"):
    with driver.session() as session:
        examples = session.read_transaction(get_person_of_bill, node_label, relation, node_label_2, element_id)
        # Output the examples
        if not examples:
            person_value = "NA"
        else:
            person_value = examples[0][node_label]
    return person_value

In [134]:
df_bill_canton["Person"] = df_bill_canton["Bill"].apply(lambda l: get_person_of_bill_main(l["element_id"]))

/tmp/ipykernel_8066/306973483.py:13: DeprecationWarning: read_transaction has been renamed to execute_read
  examples = session.read_transaction(get_person_of_bill, node_label, relation, node_label_2, element_id)


In [135]:
df_bill_canton

,Bill,Canton,Tag,BillText,Person
0,{'element_id': '4:57021189-84a0-4dfc-8fa3-eb88...,"{'name': 'Schwyz', 'abbrev': 'SZ'}",Migration,<text><p>Gestützt auf Artikel 160 Absatz 1 der...,NA
1,{'element_id': '4:57021189-84a0-4dfc-8fa3-eb88...,"{'name': 'Schwyz', 'abbrev': 'SZ'}",Landwirtschaft,"<p><b>Debatte im Nationalrat, 23.09.2015</b></...",NA
2,{'element_id': '4:57021189-84a0-4dfc-8fa3-eb88...,"{'name': 'Schwyz', 'abbrev': 'SZ'}",Finanzwesen,<text><p>Gestützt auf Artikel 160 Absatz 1 der...,NA
3,{'element_id': '4:57021189-84a0-4dfc-8fa3-eb88...,"{'name': 'Schwyz', 'abbrev': 'SZ'}",Europapolitik,<text><p>Gestützt auf Artikel 160 Absatz 1 der...,NA
4,{'element_id': '4:57021189-84a0-4dfc-8fa3-eb88...,"{'name': 'Aargau', 'abbrev': 'AG'}",Staatspolitik,<text><p>a. Anhebung der Integrationspauschale...,NA
...,...,...,...,...,...
435,{'element_id': '4:57021189-84a0-4dfc-8fa3-eb88...,"{'name': 'Tessin', 'abbrev': 'TI'}",Kultur,<text><p>Gestützt auf Artikel 160 Absatz 1 der...,NA
436,{'element_id': '4:57021189-84a0-4dfc-8fa3-eb88...,"{'name': 'Tessin', 'abbrev': 'TI'}",Finanzwesen,<p>Der Grosse Rat von Republik und Kanton Tess...,NA
437,{'element_id': '4:57021189-84a0-4dfc-8fa3-eb88...,"{'name': 'Tessin', 'abbrev': 'TI'}",Recht Allgemein,<text><p>Gestützt auf Artikel 160 Absatz 1 der...,NA
438,{'element_id': '4:57021189-84a0-4dfc-8fa3-eb88...,"{'name': 'Tessin', 'abbrev': 'TI'}",Recht Allgemein,<text><p>Gestützt auf Artikel 160 Absatz 1 der...,NA


In [137]:
set(df_bill_canton["Person"])

{'NA'}

In [141]:
def get_bill_person(tx, node_label, relation, node_label_2, eli):
    query = """
    MATCH (b:{})-[r:{}]->(rdata:{})
    WHERE elementId(rdata) = '{}'
    RETURN b, rdata LIMIT 10
    """.format(node_label, relation, node_label_2, eli)
    result = tx.run(query)
    return [{node_label: record["b"], node_label_2: record["rdata"]} for record in result]

# Main block to run the query
def get_bill_person_main(eli, node_label="Person", relation="SPONSORS", node_label_2="Bill"):
    with driver.session() as session:
        examples = session.read_transaction(get_bill_person, node_label, relation, node_label_2, eli)
        for example in examples:
            print("Node1 {} data".format(node_label), example[node_label])
            print("{}, Node2 {} data".format(relation, node_label_2), example[node_label_2])
            print("-" * 40)

In [142]:
get_bill_person_main("4:57021189-84a0-4dfc-8fa3-eb8852328616:454275")

/tmp/ipykernel_8066/2093912536.py:13: DeprecationWarning: read_transaction has been renamed to execute_read
  examples = session.read_transaction(get_bill_person, node_label, relation, node_label_2, eli)


Node1 Person data <Node element_id='4:57021189-84a0-4dfc-8fa3-eb8852328616:585934' labels=frozenset({'Unique Person', 'Person'}) properties={'date_birth': neo4j.time.Date(1969, 5, 5), 'uid': 3888, 'gender': 'm', 'last_name': 'Hiltpold', 'first_name': 'Hugues'}>
SPONSORS, Node2 Bill data <Node element_id='4:57021189-84a0-4dfc-8fa3-eb8852328616:454275' labels=frozenset({'Question Time', 'Bill'}) properties={'title_de': 'Kyoto-Protokoll. Bilanz der bisherigen Politik?', 'bill_number': '09.5595', 'title_fr': 'Protocole de Kyoto. Bilan de la politique menée?', 'urgent': False, 'status': 'Resolved'}>
----------------------------------------


In [144]:
get_bill_person_main("4:57021189-84a0-4dfc-8fa3-eb8852328616:693723")

/tmp/ipykernel_8066/2093912536.py:13: DeprecationWarning: read_transaction has been renamed to execute_read
  examples = session.read_transaction(get_bill_person, node_label, relation, node_label_2, eli)


In [143]:
df_bill_canton["Bill"][0]["element_id"]

'4:57021189-84a0-4dfc-8fa3-eb8852328616:693723'

In [79]:
df = get_bill_with_specific_about_tag_main()
df["Bill"] = df["Bill"].apply(lambda l : {"element_id": l.element_id, "bill_number": l['bill_number'], "title_fr": l['title_fr'], "title_de": l['title_de']})
df["Tag"] = df["Tag"].apply(lambda l : l['name'])

/tmp/ipykernel_8066/837977075.py:153: DeprecationWarning: read_transaction has been renamed to execute_read
  examples = session.read_transaction(get_bill_with_specific_about_tag, node_label, relation, node_label_2)


Node1 Bill data <Node element_id='4:57021189-84a0-4dfc-8fa3-eb8852328616:494770' labels=frozenset({'Question Time', 'Bill'}) properties={'title_de': 'Was hält der Bundesrat vom BAG-Vorschlag betreffend Auslandpreisvergleich von Medikamenten?', 'bill_number': '12.5057', 'title_fr': "Comparaison avec l'étranger du prix des médicaments. Proposition de l'OFSP", 'urgent': False, 'status': 'Resolved'}>
ABOUT, Node2 Tag data <Node element_id='4:57021189-84a0-4dfc-8fa3-eb8852328616:454255' labels=frozenset({'Tag', 'Issuelist'}) properties={'name': 'Gesundheit'}>


In [88]:
get_node_data_with_relation_and_node_data_main(node_label="Canton", relation="SPONSORS", node_label_2="Bill")

/tmp/ipykernel_8066/837977075.py:133: DeprecationWarning: read_transaction has been renamed to execute_read
  examples = session.read_transaction(get_node_data_with_relation_and_node, node_label, relation, node_label_2)


Node1 Canton data <Node element_id='4:57021189-84a0-4dfc-8fa3-eb8852328616:518263' labels=frozenset({'Canton', 'Location'}) properties={'name': 'Schwyz', 'abbrev': 'SZ'}>
SPONSORS, Node2 Bill data <Node element_id='4:57021189-84a0-4dfc-8fa3-eb8852328616:693723' labels=frozenset({'Cantonal Initiative', 'Bill'}) properties={'title_de': 'Einbürgerungsverfahren', 'bill_number': '03.317', 'title_fr': 'Procédure de naturalisation', 'urgent': False, 'status': 'Resolved'}>
----------------------------------------
Node1 Canton data <Node element_id='4:57021189-84a0-4dfc-8fa3-eb8852328616:518263' labels=frozenset({'Canton', 'Location'}) properties={'name': 'Schwyz', 'abbrev': 'SZ'}>
SPONSORS, Node2 Bill data <Node element_id='4:57021189-84a0-4dfc-8fa3-eb8852328616:480233' labels=frozenset({'Cantonal Initiative', 'Bill'}) properties={'title_de': 'Umsetzbares revidiertes Gewässerschutzgesetz', 'bill_number': '12.309', 'title_fr': 'Loi fédérale sur la protection des eaux. Révision', 'urgent': False

In [81]:
set(df["Tag"])

{'Gesundheit', 'Wirtschaft'}

In [82]:
df.head()

,Bill,Tag
0,{'element_id': '4:57021189-84a0-4dfc-8fa3-eb88...,Gesundheit
1,{'element_id': '4:57021189-84a0-4dfc-8fa3-eb88...,Gesundheit
2,{'element_id': '4:57021189-84a0-4dfc-8fa3-eb88...,Gesundheit
3,{'element_id': '4:57021189-84a0-4dfc-8fa3-eb88...,Gesundheit
4,{'element_id': '4:57021189-84a0-4dfc-8fa3-eb88...,Gesundheit


In [101]:
df["Bill"][1]

{'element_id': '4:57021189-84a0-4dfc-8fa3-eb8852328616:731227',
 'bill_number': '08.3464',
 'title_fr': 'Tarif des analyses version bêta',
 'title_de': 'Labortarife in der Beta-Version der Analysenliste'}

In [84]:
get_unique_relationships_with_nodes_main()

/tmp/ipykernel_8066/837977075.py:57: DeprecationWarning: read_transaction has been renamed to execute_read
  unique_relationships = session.read_transaction(get_unique_relationships_with_nodes, node_label)


Unique relationships related to 'Bill' nodes:
('SUBMITTED_TO', ['Chamber'])
('CONTAINS', ['BillText', 'Submitted'])
('ABOUT', ['Tag', 'Issuelist'])
('CONTAINS', ['Debate', 'Chamber Debate'])
('PROPOSED_TO', ['Department'])
('CONTAINS', ['Debate', 'Committee Debate'])
('CONTAINS', ['BillText', 'Reasoning'])
('RELATED_TO', ['Bill', 'Motion'])
('CONTAINS', ['Enactment Draft'])
('CONTAINS', ['BillText', 'News Report'])
('CONTAINS', ['BillText', 'Media Summary'])
('RELATED_TO', ['Bill', 'Parliamentary Initiative'])
('CONTAINS', ['URL'])
('RELATED_TO', ['Bill', 'Federal Council Dispatch'])
('RELATED_TO', ['Bill', 'Postulate'])
('RELATED_TO', ['Bill', 'Federal Council Report'])
('RELATED_TO', ['Bill', 'Interpellation'])
('RELATED_TO', ['Bill', 'Cantonal Initiative'])
('RELATED_TO', ['Bill', 'Question'])
('CONTAINS', ['URL', 'Popular vote'])
('CONTAINS', ['URL', 'Negotiation booklet'])
('CONTAINS', ['URL', 'Commission documents'])
('CONTAINS', ['URL', 'Consultation'])
('CONTAINS', ['URL', 'Mes

In [85]:
get_unique_relationships_with_nodes_main("Canton")

/tmp/ipykernel_8066/837977075.py:57: DeprecationWarning: read_transaction has been renamed to execute_read
  unique_relationships = session.read_transaction(get_unique_relationships_with_nodes, node_label)


Unique relationships related to 'Canton' nodes:
('SUBMITTED_TO', ['Chamber'])
('CONTAINS', ['BillText', 'Submitted'])
('ABOUT', ['Tag', 'Issuelist'])
('CONTAINS', ['Debate', 'Chamber Debate'])
('PROPOSED_TO', ['Department'])
('CONTAINS', ['Debate', 'Committee Debate'])
('CONTAINS', ['BillText', 'Reasoning'])
('RELATED_TO', ['Bill', 'Motion'])
('CONTAINS', ['Enactment Draft'])
('CONTAINS', ['BillText', 'News Report'])
('CONTAINS', ['BillText', 'Media Summary'])
('RELATED_TO', ['Bill', 'Parliamentary Initiative'])
('CONTAINS', ['URL'])
('RELATED_TO', ['Bill', 'Federal Council Dispatch'])
('RELATED_TO', ['Bill', 'Postulate'])
('RELATED_TO', ['Bill', 'Federal Council Report'])
('RELATED_TO', ['Bill', 'Interpellation'])
('RELATED_TO', ['Bill', 'Cantonal Initiative'])
('RELATED_TO', ['Bill', 'Question'])
('CONTAINS', ['URL', 'Popular vote'])
('CONTAINS', ['URL', 'Negotiation booklet'])
('CONTAINS', ['URL', 'Commission documents'])
('CONTAINS', ['URL', 'Consultation'])
('CONTAINS', ['URL', 'M

In [83]:
get_unique_node_labels()

{'Ambiguous Employer',
 'Ambiguous Person',
 'Bill',
 'BillText',
 'Canton',
 'Cantonal Initiative',
 'Chamber',
 'Chamber Debate',
 'City',
 'Club',
 'Commission',
 'Commission documents',
 'Committee',
 'Committee Debate',
 'Consultation',
 'Cooperative',
 'Corporation',
 'Debate',
 'Delegation',
 'Department',
 'Discussion',
 'Employer',
 'Enactment Draft',
 'External Entity',
 'Federal Commission',
 'Federal Council Dispatch',
 'Federal Council Report',
 'Federal Response',
 'Foundation',
 'General Partnership',
 'Inconsistent Timestamp',
 'Interest Group',
 'Interpellation',
 'Issuelist',
 'Legislative Period',
 'Legislative Proposition',
 'Limited Liability Company',
 'Limited Partnership',
 'Location',
 'Media Summary',
 'Meeting',
 'Message/Report not in BBl',
 'Motion',
 'MultiDebate',
 'Negotiation booklet',
 'News Report',
 'Not Bill Related',
 'Organisation',
 'Other Parliamentary Activities',
 'Parliamentary Group',
 'Parliamentary Initiative',
 'Partition',
 'Party',
 'Pa

In [19]:
get_bill_with_specific_about_tag_main()

/tmp/ipykernel_8066/2982058914.py:152: DeprecationWarning: read_transaction has been renamed to execute_read
  examples = session.read_transaction(get_node_data_with_relation_and_node, node_label, relation, node_label_2)


Node1 Bill data <Node element_id='4:57021189-84a0-4dfc-8fa3-eb8852328616:494770' labels=frozenset({'Question Time', 'Bill'}) properties={'title_de': 'Was hält der Bundesrat vom BAG-Vorschlag betreffend Auslandpreisvergleich von Medikamenten?', 'bill_number': '12.5057', 'title_fr': "Comparaison avec l'étranger du prix des médicaments. Proposition de l'OFSP", 'urgent': False, 'status': 'Resolved'}>
ABOUT, Node2 Tag data <Node element_id='4:57021189-84a0-4dfc-8fa3-eb8852328616:454255' labels=frozenset({'Tag', 'Issuelist'}) properties={'name': 'Gesundheit'}>
----------------------------------------
Node1 Bill data <Node element_id='4:57021189-84a0-4dfc-8fa3-eb8852328616:731227' labels=frozenset({'Bill', 'Interpellation'}) properties={'title_de': 'Labortarife in der Beta-Version der Analysenliste', 'bill_number': '08.3464', 'title_fr': 'Tarif des analyses version bêta', 'urgent': False, 'status': 'Resolved'}>
ABOUT, Node2 Tag data <Node element_id='4:57021189-84a0-4dfc-8fa3-eb8852328616:454

In [9]:
tags = get_node_data_main(node_label="Tag")
len(tags)

<Node element_id='4:57021189-84a0-4dfc-8fa3-eb8852328616:454255' labels=frozenset({'Tag', 'Issuelist'}) properties={'name': 'Gesundheit'}>
----------------------------------------
<Node element_id='4:57021189-84a0-4dfc-8fa3-eb8852328616:454256' labels=frozenset({'Tag', 'Issuelist'}) properties={'name': 'Raumplanung und Wohnungswesen'}>
----------------------------------------
<Node element_id='4:57021189-84a0-4dfc-8fa3-eb8852328616:454257' labels=frozenset({'Tag', 'Issuelist'}) properties={'name': 'Medien und Kommunikation'}>
----------------------------------------
<Node element_id='4:57021189-84a0-4dfc-8fa3-eb8852328616:454258' labels=frozenset({'Tag', 'Issuelist'}) properties={'name': 'Landwirtschaft'}>
----------------------------------------
<Node element_id='4:57021189-84a0-4dfc-8fa3-eb8852328616:454259' labels=frozenset({'Tag', 'Issuelist'}) properties={'name': 'Kultur'}>
----------------------------------------
<Node element_id='4:57021189-84a0-4dfc-8fa3-eb8852328616:454260' la

/tmp/ipykernel_8066/327362110.py:74: DeprecationWarning: read_transaction has been renamed to execute_read
  examples = session.read_transaction(get_node_data, node_label)


28

In [17]:
for t in tags:
    print(t["name"])

Gesundheit
Raumplanung und Wohnungswesen
Medien und Kommunikation
Landwirtschaft
Kultur
Finanzwesen
Wirtschaft
Recht Allgemein
Verkehr
Parlament
Europapolitik
Migration
Bildung
Staatspolitik
Sicherheitspolitik
Umwelt
Soziale Fragen
Internationale Politik
Energie
Wissenschaft und Forschung
Zivilrecht
Menschenrechte
Internationales Recht
Steuer
Sozialer Schutz
Strafrecht
Beschäftigung und Arbeit
Gerichtswesen


In [56]:
get_unique_node_labels()

{'Ambiguous Employer',
 'Ambiguous Person',
 'Bill',
 'BillText',
 'Canton',
 'Cantonal Initiative',
 'Chamber',
 'Chamber Debate',
 'City',
 'Club',
 'Commission',
 'Commission documents',
 'Committee',
 'Committee Debate',
 'Consultation',
 'Cooperative',
 'Corporation',
 'Debate',
 'Delegation',
 'Department',
 'Discussion',
 'Employer',
 'Enactment Draft',
 'External Entity',
 'Federal Commission',
 'Federal Council Dispatch',
 'Federal Council Report',
 'Federal Response',
 'Foundation',
 'General Partnership',
 'Inconsistent Timestamp',
 'Interest Group',
 'Interpellation',
 'Issuelist',
 'Legislative Period',
 'Legislative Proposition',
 'Limited Liability Company',
 'Limited Partnership',
 'Location',
 'Media Summary',
 'Meeting',
 'Message/Report not in BBl',
 'Motion',
 'MultiDebate',
 'Negotiation booklet',
 'News Report',
 'Not Bill Related',
 'Organisation',
 'Other Parliamentary Activities',
 'Parliamentary Group',
 'Parliamentary Initiative',
 'Partition',
 'Party',
 'Pa

In [39]:
get_node_data_with_relation_and_node_data_main()

Node1 Bill data <Node element_id='4:fa996e42-3a3f-4866-ad85-cb06ad63f099:494770' labels=frozenset({'Bill', 'Question Time'}) properties={'title_de': 'Was hält der Bundesrat vom BAG-Vorschlag betreffend Auslandpreisvergleich von Medikamenten?', 'bill_number': '12.5057', 'title_fr': "Comparaison avec l'étranger du prix des médicaments. Proposition de l'OFSP", 'urgent': False, 'status': 'Resolved'}>
ABOUT, Node2 Tag data <Node element_id='4:fa996e42-3a3f-4866-ad85-cb06ad63f099:454255' labels=frozenset({'Tag', 'Issuelist'}) properties={'name': 'Gesundheit'}>
----------------------------------------
Node1 Bill data <Node element_id='4:fa996e42-3a3f-4866-ad85-cb06ad63f099:731227' labels=frozenset({'Interpellation', 'Bill'}) properties={'title_de': 'Labortarife in der Beta-Version der Analysenliste', 'bill_number': '08.3464', 'title_fr': 'Tarif des analyses version bêta', 'urgent': False, 'status': 'Resolved'}>
ABOUT, Node2 Tag data <Node element_id='4:fa996e42-3a3f-4866-ad85-cb06ad63f099:454

/tmp/ipykernel_7626/893329818.py:110: DeprecationWarning: read_transaction has been renamed to execute_read
  examples = session.read_transaction(get_node_data_with_relation_and_node, node_label, relation, node_label_2)


In [12]:
get_unique_node_labels_main()

Unexpected Vote Total
Limited Partnership
Transcript
Bill
Public Foundation
Message/Report not in BBl
Issuelist
Question
General Partnership
Federal Council Report
Reasoning
Person
Consultation
Limited Liability Company
Discussion
Parliamentary Group
Chamber
Committee
Cooperative
Corporation
Popular vote
Meeting
Federal Council Dispatch
Committee Debate
Canton
Negotiation booklet
Location
Postprocessing Necessary
Written Response
Media Summary
Year
Proposal
Vote
Petition
Session
Debate
Interpellation
Federal Response
Not Bill Related
Cantonal Initiative
Legislative Period
Unique Person
Commission documents
Club
Public Institute
Postulate
Submitted
External Entity
Source
Tag
Interest Group
Organisation
BillText
Party
Partition
Commission
Ambiguous Person
Other Parliamentary Activities
Ambiguous Employer
Delegation
Foundation
Question Time
News Report
Federal Commission
URL
Chamber Debate
Legislative Proposition
Enactment Draft
Parliamentary Initiative
Public Body
Speech
MultiDebate
Inco

In [9]:
get_unique_relationships_main()

/tmp/ipykernel_7626/2236063300.py:22: DeprecationWarning: read_transaction has been renamed to execute_read
  unique_relationships = session.read_transaction(get_unique_relationships, node_label)


Unique relationships related to 'Bill' nodes:
SPONSORS
SUBMITTED_TO
CONTAINS
ABOUT
REACTION_TO
PROPOSED_TO
COSPONSORS
RELATED_TO
ACTING_AS
PROPOSED


In [10]:
get_unique_relationships_main("BillText")

/tmp/ipykernel_7626/2236063300.py:22: DeprecationWarning: read_transaction has been renamed to execute_read
  unique_relationships = session.read_transaction(get_unique_relationships, node_label)


Unique relationships related to 'BillText' nodes:
CONTAINS


In [15]:
get_unique_relationships_with_nodes_main()

/tmp/ipykernel_7626/1527625206.py:56: DeprecationWarning: read_transaction has been renamed to execute_read
  unique_relationships = session.read_transaction(get_unique_relationships_with_nodes, node_label)


Unique relationships related to 'Bill' nodes:
('SUBMITTED_TO', ['Chamber'])
('CONTAINS', ['BillText', 'Submitted'])
('ABOUT', ['Tag', 'Issuelist'])
('CONTAINS', ['Debate', 'Chamber Debate'])
('PROPOSED_TO', ['Department'])
('CONTAINS', ['Debate', 'Committee Debate'])
('CONTAINS', ['BillText', 'Reasoning'])
('RELATED_TO', ['Bill', 'Motion'])
('CONTAINS', ['Enactment Draft'])
('CONTAINS', ['BillText', 'News Report'])
('CONTAINS', ['BillText', 'Media Summary'])
('RELATED_TO', ['Bill', 'Parliamentary Initiative'])
('CONTAINS', ['URL'])
('RELATED_TO', ['Bill', 'Federal Council Dispatch'])
('RELATED_TO', ['Bill', 'Postulate'])
('RELATED_TO', ['Bill', 'Federal Council Report'])
('RELATED_TO', ['Bill', 'Interpellation'])
('RELATED_TO', ['Bill', 'Cantonal Initiative'])
('RELATED_TO', ['Bill', 'Question'])
('CONTAINS', ['URL', 'Popular vote'])
('CONTAINS', ['URL', 'Negotiation booklet'])
('CONTAINS', ['URL', 'Commission documents'])
('CONTAINS', ['URL', 'Consultation'])
('CONTAINS', ['URL', 'Mes

In [16]:
get_unique_relationships_with_nodes_main("Speech")

/tmp/ipykernel_7626/1527625206.py:56: DeprecationWarning: read_transaction has been renamed to execute_read
  unique_relationships = session.read_transaction(get_unique_relationships_with_nodes, node_label)


Unique relationships related to 'Speech' nodes:
('SUBMITTED_TO', ['Chamber'])
('CONTAINS', ['BillText', 'Submitted'])
('ABOUT', ['Tag', 'Issuelist'])
('CONTAINS', ['Debate', 'Chamber Debate'])
('PROPOSED_TO', ['Department'])
('CONTAINS', ['Debate', 'Committee Debate'])
('CONTAINS', ['BillText', 'Reasoning'])
('RELATED_TO', ['Bill', 'Motion'])
('CONTAINS', ['Enactment Draft'])
('CONTAINS', ['BillText', 'News Report'])
('CONTAINS', ['BillText', 'Media Summary'])
('RELATED_TO', ['Bill', 'Parliamentary Initiative'])
('CONTAINS', ['URL'])
('RELATED_TO', ['Bill', 'Federal Council Dispatch'])
('RELATED_TO', ['Bill', 'Postulate'])
('RELATED_TO', ['Bill', 'Federal Council Report'])
('RELATED_TO', ['Bill', 'Interpellation'])
('RELATED_TO', ['Bill', 'Cantonal Initiative'])
('RELATED_TO', ['Bill', 'Question'])
('CONTAINS', ['URL', 'Popular vote'])
('CONTAINS', ['URL', 'Negotiation booklet'])
('CONTAINS', ['URL', 'Commission documents'])
('CONTAINS', ['URL', 'Consultation'])
('CONTAINS', ['URL', 'M

In [17]:
get_unique_relationships_with_nodes_main("Transcript")

/tmp/ipykernel_7626/1527625206.py:56: DeprecationWarning: read_transaction has been renamed to execute_read
  unique_relationships = session.read_transaction(get_unique_relationships_with_nodes, node_label)


Unique relationships related to 'Transcript' nodes:
('SUBMITTED_TO', ['Chamber'])
('CONTAINS', ['BillText', 'Submitted'])
('ABOUT', ['Tag', 'Issuelist'])
('CONTAINS', ['Debate', 'Chamber Debate'])
('PROPOSED_TO', ['Department'])
('CONTAINS', ['Debate', 'Committee Debate'])
('CONTAINS', ['BillText', 'Reasoning'])
('RELATED_TO', ['Bill', 'Motion'])
('CONTAINS', ['Enactment Draft'])
('CONTAINS', ['BillText', 'News Report'])
('CONTAINS', ['BillText', 'Media Summary'])
('RELATED_TO', ['Bill', 'Parliamentary Initiative'])
('CONTAINS', ['URL'])
('RELATED_TO', ['Bill', 'Federal Council Dispatch'])
('RELATED_TO', ['Bill', 'Postulate'])
('RELATED_TO', ['Bill', 'Federal Council Report'])
('RELATED_TO', ['Bill', 'Interpellation'])
('RELATED_TO', ['Bill', 'Cantonal Initiative'])
('RELATED_TO', ['Bill', 'Question'])
('CONTAINS', ['URL', 'Popular vote'])
('CONTAINS', ['URL', 'Negotiation booklet'])
('CONTAINS', ['URL', 'Commission documents'])
('CONTAINS', ['URL', 'Consultation'])
('CONTAINS', ['URL'

In [5]:
def get_bill_about_data(tx):
    query = """
    MATCH (b:Bill)-[r:ABOUT]->(aboutData)
    RETURN b, aboutData LIMIT 10
    """
    result = tx.run(query)
    return [{"bill": record["b"], "about_data": record["aboutData"]} for record in result]

# Main block to run the query
with driver.session() as session:
    bill_about_examples = session.read_transaction(get_bill_about_data)
    
    # Output the examples
    print("Examples of 'Bill' nodes with 'ABOUT' relationship data:")
    for example in bill_about_examples:
        print(f"Bill: {example['bill']}")
        print(f"About Data: {example['about_data']}")
        print("-" * 40)

Examples of 'Bill' nodes with 'ABOUT' relationship data:
Bill: <Node element_id='4:fa996e42-3a3f-4866-ad85-cb06ad63f099:454275' labels=frozenset({'Question Time', 'Bill'}) properties={'title_de': 'Kyoto-Protokoll. Bilanz der bisherigen Politik?', 'bill_number': '09.5595', 'title_fr': 'Protocole de Kyoto. Bilan de la politique menée?', 'urgent': False, 'status': 'Resolved'}>
About Data: <Node element_id='4:fa996e42-3a3f-4866-ad85-cb06ad63f099:454270' labels=frozenset({'Issuelist', 'Tag'}) properties={'name': 'Umwelt'}>
----------------------------------------
Bill: <Node element_id='4:fa996e42-3a3f-4866-ad85-cb06ad63f099:454276' labels=frozenset({'Question Time', 'Bill'}) properties={'title_de': 'Einsparungspotenzial bei der SRG', 'bill_number': '09.5612', 'title_fr': "Possibilités d'économies pour la SSR", 'urgent': False, 'status': 'Resolved'}>
About Data: <Node element_id='4:fa996e42-3a3f-4866-ad85-cb06ad63f099:454257' labels=frozenset({'Issuelist', 'Tag'}) properties={'name': 'Medie

/tmp/ipykernel_7505/2839504859.py:10: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
/tmp/ipykernel_7505/2839504859.py:11: DeprecationWarning: read_transaction has been renamed to execute_read
  bill_about_examples = session.read_transaction(get_bill_about_data)


In [3]:
class Neo4jQuery:

    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self.driver.close()

    def get_relations_for_bill_nodes(self):
        query = """
        MATCH (b:Bill)-[r]->(n)
        RETURN b, r, n
        LIMIT 25
        UNION
        MATCH (n)-[r]->(b:Bill)
        RETURN n, r, b
        LIMIT 25
        """
        with self.driver.session() as session:
            result = session.run(query)
            return [(record['b'], record['r'], record['n']) for record in result]

# Create a Neo4jQuery instance
neo4j_query = Neo4jQuery(uri, username, password)

# Get relations for Bill nodes
relations = neo4j_query.get_relations_for_bill_nodes()

# Display the results
for relation in relations:
    print(f"Node 1: {relation[0]} - Relationship: {relation[1]} - Node 2: {relation[2]}")


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: All subqueries in a UNION [ALL] should have the same ordering for the return columns. Using differently ordered return items in a UNION [ALL] clause is deprecated and will be removed in a future version.} {position: line: 2, column: 9, offset: 9} for query: '\n        MATCH (b:Bill)-[r]->(n)\n        RETURN b, r, n\n        LIMIT 25\n        UNION\n        MATCH (n)-[r]->(b:Bill)\n        RETURN n, r, b\n        LIMIT 25\n        '


Node 1: <Node element_id='4:fa996e42-3a3f-4866-ad85-cb06ad63f099:454275' labels=frozenset({'Question Time', 'Bill'}) properties={'title_de': 'Kyoto-Protokoll. Bilanz der bisherigen Politik?', 'bill_number': '09.5595', 'title_fr': 'Protocole de Kyoto. Bilan de la politique menée?', 'urgent': False, 'status': 'Resolved'}> - Relationship: <Relationship element_id='5:fa996e42-3a3f-4866-ad85-cb06ad63f099:492646' nodes=(<Node element_id='4:fa996e42-3a3f-4866-ad85-cb06ad63f099:454275' labels=frozenset({'Question Time', 'Bill'}) properties={'title_de': 'Kyoto-Protokoll. Bilanz der bisherigen Politik?', 'bill_number': '09.5595', 'title_fr': 'Protocole de Kyoto. Bilan de la politique menée?', 'urgent': False, 'status': 'Resolved'}>, <Node element_id='4:fa996e42-3a3f-4866-ad85-cb06ad63f099:518450' labels=frozenset({'Chamber'}) properties={'name': 'Nationalrat', 'abbrev': 'NR'}>) type='SUBMITTED_TO' properties={'date': neo4j.time.Date(2009, 12, 7)}> - Node 2: <Node element_id='4:fa996e42-3a3f-4866

In [1]:
from neo4j import GraphDatabase

# Define the connection URI and authentication
uri = "bolt://94.230.217.43:7687"  # Update this if you're using a different address   localhost
username = "neo4j"  # Your Neo4j username
password = "lagginhorn"  # Your Neo4j password   newpassword

# Create a Neo4j driver instance
driver = GraphDatabase.driver(uri, auth=(username, password))

# Define a function to query the Neo4j database
def query_neo4j(query):
    with driver.session() as session:
        # Run the query and return the results
        result = session.run(query)
        return [record for record in result]

# Example Cypher query to retrieve all nodes and relationships
cypher_query = """
MATCH (n)-[r]->(m)
RETURN n, r, m
LIMIT 25
"""

# Execute the query and print the results
results = query_neo4j(cypher_query)
for record in results:
    print(record)

# Close the driver connection when done
driver.close()


<Record n=<Node element_id='4:fa996e42-3a3f-4866-ad85-cb06ad63f099:170765' labels=frozenset({'Debate', 'Chamber Debate'}) properties={'uid': '12_5127.1', 'date_start': neo4j.time.Date(2012, 3, 12), 'date_end': neo4j.time.Date(2012, 3, 12), 'order_stamp': 1}> r=<Relationship element_id='5:fa996e42-3a3f-4866-ad85-cb06ad63f099:0' nodes=(<Node element_id='4:fa996e42-3a3f-4866-ad85-cb06ad63f099:170765' labels=frozenset({'Debate', 'Chamber Debate'}) properties={'uid': '12_5127.1', 'date_start': neo4j.time.Date(2012, 3, 12), 'date_end': neo4j.time.Date(2012, 3, 12), 'order_stamp': 1}>, <Node element_id='4:fa996e42-3a3f-4866-ad85-cb06ad63f099:518450' labels=frozenset({'Chamber'}) properties={'name': 'Nationalrat', 'abbrev': 'NR'}>) type='TAKES_PLACE_IN' properties={}> m=<Node element_id='4:fa996e42-3a3f-4866-ad85-cb06ad63f099:518450' labels=frozenset({'Chamber'}) properties={'name': 'Nationalrat', 'abbrev': 'NR'}>>
<Record n=<Node element_id='4:fa996e42-3a3f-4866-ad85-cb06ad63f099:170615' labe

In [15]:
def get_bills_nodes(driver):
    with driver.session() as session:
        # Cypher query to fetch all Bills nodes
        result = session.run("MATCH (b:Bill) RETURN b LIMIT 25")
        bills = [record["b"] for record in result]
        return bills

driver = GraphDatabase.driver(uri, auth=(username, password))

# Fetch and print the Bills nodes
bills_nodes = get_bills_nodes(driver)
for bill in bills_nodes:
    print(bill)

<Node element_id='4:fa996e42-3a3f-4866-ad85-cb06ad63f099:454275' labels=frozenset({'Bill', 'Question Time'}) properties={'title_de': 'Kyoto-Protokoll. Bilanz der bisherigen Politik?', 'bill_number': '09.5595', 'title_fr': 'Protocole de Kyoto. Bilan de la politique menée?', 'urgent': False, 'status': 'Resolved'}>
<Node element_id='4:fa996e42-3a3f-4866-ad85-cb06ad63f099:454276' labels=frozenset({'Bill', 'Question Time'}) properties={'title_de': 'Einsparungspotenzial bei der SRG', 'bill_number': '09.5612', 'title_fr': "Possibilités d'économies pour la SSR", 'urgent': False, 'status': 'Resolved'}>
<Node element_id='4:fa996e42-3a3f-4866-ad85-cb06ad63f099:454277' labels=frozenset({'Bill', 'Question Time'}) properties={'title_de': 'Putsch in Honduras', 'bill_number': '09.5570', 'title_fr': "Coup d'Etat au Honduras", 'urgent': False, 'status': 'Resolved'}>
<Node element_id='4:fa996e42-3a3f-4866-ad85-cb06ad63f099:454278' labels=frozenset({'Bill', 'Question Time'}) properties={'title_de': 'Winte

In [16]:
def get_unique_node_labels():
    with driver.session() as session:
        # Cypher query to get all labels of nodes
        result = session.run("MATCH (n) RETURN DISTINCT labels(n) AS label_list")
        # Extracting and flattening the label lists
        unique_labels = set()
        for record in result:
            unique_labels.update(record["label_list"])
        return unique_labels

# Fetch and print the unique node labels
unique_labels = get_unique_node_labels()
for label in unique_labels:
    print(label)

Chamber Debate
Legislative Period
Committee
Foundation
Federal Council Report
Postulate
Commission
Federal Council Dispatch
Canton
Simple Cooperative
Not Bill Related
Vote
External Entity
Corporation
Popular vote
Legislative Proposition
Discussion
Partition
Federal Response
Other Parliamentary Activities
Club
Message/Report not in BBl
Parliamentary Initiative
BillText
Committee Debate
City
URL
Department
Petition
Cooperative
Limited Liability Company
Public Body
Party Class
Unique Person
Ambiguous Person
Written Response
Year
Unexpected Vote Total
Consultation
Speech
Federal Commission
Session
Interest Group
Public Foundation
Question Time
Public Institute
Limited Partnership
Media Summary
Location
Person
Reasoning
News Report
Commission documents
Question
Ambiguous Employer
Transcript
Cantonal Initiative
Negotiation booklet
MultiDebate
Bill
Enactment Draft
Submitted
Party
Proposal
Interpellation
Organisation
Postprocessing Necessary
Inconsistent Timestamp
Parliamentary Group
General P

In [27]:
def get_bills_nodes(driver):
    with driver.session() as session:
        # Cypher query to fetch all Bills nodes
        result = session.run("MATCH (b:BillText) RETURN b LIMIT 50")
        bills = [record["b"] for record in result]
        return bills

driver = GraphDatabase.driver(uri, auth=(username, password))

# Fetch and print the Bills nodes
bills_nodes = get_bills_nodes(driver)
for bill in bills_nodes:
    print(bill)

<Node element_id='4:fa996e42-3a3f-4866-ad85-cb06ad63f099:170838' labels=frozenset({'BillText', 'Submitted'}) properties={'uid': '12.5206.bt.45421f8c-f6c6-42cb-9d86-1e2873c67a69', 'text': '<p>Vor allem im Umfeld von Asylunterkünften kommt es zu immer mehr Belästigungen, Tätlichkeiten, Diebstählen, Einbrüchen, Drogendelikten und anderen Straftaten, die durch renitente und kriminelle Asylanten verursacht werden.</p><p>Ist der Bundesrat auch der Meinung, dass Asylsuchende, welche die öffentliche Ordnung und Sicherheit gefährden, zu internieren und aus dem Asylverfahren auszuschliessen sind?</p>'}>
<Node element_id='4:fa996e42-3a3f-4866-ad85-cb06ad63f099:170839' labels=frozenset({'BillText', 'Submitted'}) properties={'uid': '12.5319.bt.8f9d1768-16fb-4a7f-80f9-74f7b9c45a2f', 'text': '<p>- Welche Konsequenzen hätte ein Auseinanderbrechen des Euros für die Schweizer Volkswirtschaft?</p><p>- Hat der Bundesrat dafür ein Notfallszenario vorbereitet?</p>'}>
<Node element_id='4:fa996e42-3a3f-4866-a

In [5]:
from neo4j import GraphDatabase

# Define the connection URI and authentication
uri = "bolt://localhost:7687"  # Update this if you're using a different address
username = "neo4j"  # Your Neo4j username
password = "newpassword"  # Your Neo4j password

# Create a Neo4j driver instance
driver = GraphDatabase.driver(uri, auth=(username, password))

# Define a function to query the Neo4j database
def query_neo4j(query):
    with driver.session() as session:
        # Run the query and return the results
        result = session.run(query)
        return [record for record in result]

# Example Cypher query to retrieve all nodes and relationships
cypher_query = """
MATCH (n)-[r]->(m)
RETURN n, r, m
LIMIT 25
"""

# Execute the query and print the results
results = query_neo4j(cypher_query)
for record in results:
    print(record)

# Close the driver connection when done
driver.close()


<Record n=<Node element_id='4:57021189-84a0-4dfc-8fa3-eb8852328616:170765' labels=frozenset({'Chamber Debate', 'Debate'}) properties={'uid': '12_5127.1', 'date_start': neo4j.time.Date(2012, 3, 12), 'date_end': neo4j.time.Date(2012, 3, 12), 'order_stamp': 1}> r=<Relationship element_id='5:57021189-84a0-4dfc-8fa3-eb8852328616:0' nodes=(<Node element_id='4:57021189-84a0-4dfc-8fa3-eb8852328616:170765' labels=frozenset({'Chamber Debate', 'Debate'}) properties={'uid': '12_5127.1', 'date_start': neo4j.time.Date(2012, 3, 12), 'date_end': neo4j.time.Date(2012, 3, 12), 'order_stamp': 1}>, <Node element_id='4:57021189-84a0-4dfc-8fa3-eb8852328616:518450' labels=frozenset({'Chamber'}) properties={'name': 'Nationalrat', 'abbrev': 'NR'}>) type='TAKES_PLACE_IN' properties={}> m=<Node element_id='4:57021189-84a0-4dfc-8fa3-eb8852328616:518450' labels=frozenset({'Chamber'}) properties={'name': 'Nationalrat', 'abbrev': 'NR'}>>
<Record n=<Node element_id='4:57021189-84a0-4dfc-8fa3-eb8852328616:170615' labe

In [8]:
from neo4j import GraphDatabase

# Define the connection URI and authentication
uri = "bolt://94.230.217.43:7687"  # Update this if you're using a different address
username = "neo4j"  # Your Neo4j username
password = "lagginhorn"  # Your Neo4j password

# Create a Neo4j driver instance
driver = GraphDatabase.driver(uri, auth=(username, password))

# Define a function to query the Neo4j database
def query_neo4j(query):
    with driver.session() as session:
        # Run the query and return the results
        result = session.run(query)
        return [record for record in result]

# Example Cypher query to retrieve all nodes and relationships
cypher_query = """
MATCH (n)-[r]->(m)
RETURN n, r, m
LIMIT 25
"""

# Execute the query and print the results
results = query_neo4j(cypher_query)
for record in results:
    print(record)

# Close the driver connection when done
driver.close()


In [4]:
from neo4j import GraphDatabase

# Define the connection URI and authentication with the expired password
uri = "bolt://localhost:7687"
username = "neo4j"  # Your Neo4j username
current_password = "neo4j"  # The current password you provided
new_password = "newpassword"  # Set the new password

# Create a Neo4j driver instance
driver = GraphDatabase.driver(uri, auth=(username, current_password))

# Function to change the password
def change_password(current_password, new_password):
    try:
        with driver.session(database="system") as session:  # Connect to the system database
            # Cypher query to change the password
            query = f"ALTER CURRENT USER SET PASSWORD FROM '{current_password}' TO '{new_password}'"
            session.run(query)
            print("Password changed successfully.")
    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        driver.close()

# Change the password by executing the query
change_password(current_password, new_password)


Password changed successfully.
